In [424]:
import numpy as np

def writePitches(fn,inds,tempo=82,instrument=[0,0],add21=True,start_at= [0,0],durationsInQuarterNotes=False):
    from MidiFile import MIDIFile

    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 1   # In beats # In BPM
    volume   = 116 # 0-127, as per the MIDI standard

    ni = len(inds)
    MyMIDI = MIDIFile(ni,adjust_origin=False) # One track, defaults to format 1 (tempo track
                     # automatically created)
    MyMIDI.addTempo(track,time, tempo)


    for k in range(ni):
        MyMIDI.addProgramChange(k,k,0,instrument[k])


    times = start_at
    for k in range(len(inds)):
        channel = k
        track = k
        for i in range(len(inds[k])):
            pitch,duration,volume,isPause,tempo = inds[k][i]
            #print(pitch,duration,volume,isPause)
            track = k
            channel = k
            if not durationsInQuarterNotes:
                duration = 4*duration#*maxDurations[k] #findNearestDuration(duration*12*4)            
            #print(k,pitch,times[k],duration,100)
            if not isPause: #rest
                #print(volumes[i])
                # because of median:
                pitch = int(np.floor(pitch))
                if add21:
                    pitch += 21
                #print(pitch,times[k],duration,volume,isPause)    
                MyMIDI.addTempo(track,times[k], tempo)
                MyMIDI.addNote(track, channel, int(pitch), float(times[k]) , float(duration), int(volume))
                times[k] += duration*1.0  
            else:
                times[k] += duration*1.0
       
    with open(fn, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    print("written")  
    
bb = [57,59,62,64,67,69,72]

def plus(x,y,p):
    if x==0:
        return y
    if y==0:
        return x
    return (x+y)%p+(plus(x//p,y//p,p))*p

def lee(a,p):
    return min(a%p,(p-a)%p)

iinds = []

p = 7
n = 2
octave = [0 for k in range(p**n)]
sign = [1 for k in range(p**n)]
for b in range(p**n):
    notes = []
    for a in range(p**n):
        c = plus(a,b,p)
        digits = Integer(plus(a,b,p)).digits(p,padto=n)
        digits.reverse()
        #bar = [(bb[d]+sign[a]*octave[a]*12,2**(d+1)/sum([2**(y+1) for y in digits]),64+(-1)**d*sum(digits),False,80) for d in digits]
        S = sum([2**(y+1) for y in digits])
        bar = [(bb[d],1/2*2**(d+1)/S,32+sum(digits),c==0,80) for d in digits]
        notes.extend(bar)
        
        octave[c] = (octave[c]+c)%4
        if octave[c]==0:
            sign[c]=1
        elif octave[c]==3:
            sign[c]=-1
    iinds.append(notes)  

fn = "./midi/Zmod("+str(p)+"^"+str(n)+").mid"    
writePitches(fn,iinds,tempo=80,instrument=range(len(iinds)),add21=False,start_at= len(iinds)*[0],durationsInQuarterNotes=False)        
M = (matrix([[plus(a,b,p) for a in range(p**n)] for b in range(p**n)]))

print(M)
#plot(M).show()

written
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]
[ 1  2  3  4  5  6  0  8  9 10 11 12 13  7 15 16 17 18 19 20 14 22 23 24 25 26 27 21 29 30 31 32 33 34 28 36 37 38 39 40 41 35 43 44 45 46 47 48 42]
[ 2  3  4  5  6  0  1  9 10 11 12 13  7  8 16 17 18 19 20 14 15 23 24 25 26 27 21 22 30 31 32 33 34 28 29 37 38 39 40 41 35 36 44 45 46 47 48 42 43]
[ 3  4  5  6  0  1  2 10 11 12 13  7  8  9 17 18 19 20 14 15 16 24 25 26 27 21 22 23 31 32 33 34 28 29 30 38 39 40 41 35 36 37 45 46 47 48 42 43 44]
[ 4  5  6  0  1  2  3 11 12 13  7  8  9 10 18 19 20 14 15 16 17 25 26 27 21 22 23 24 32 33 34 28 29 30 31 39 40 41 35 36 37 38 46 47 48 42 43 44 45]
[ 5  6  0  1  2  3  4 12 13  7  8  9 10 11 19 20 14 15 16 17 18 26 27 21 22 23 24 25 33 34 28 29 30 31 32 40 41 35 36 37 38 39 47 48 42 43 44 45 46]
[ 6  0  1  2  3  4  5 13  7  8  9 10 11 12 20 14 15 16 17 18 19 27 21 22 23 24 25 26 34 28 29 30 3